# Periods Data Feature Extraction

This module help extract features from "periods_train.csv" or "periods_test.csv" such as is_holiday, wday (weekday), etc for final training of the model given an item_id.

## Note: 
This is a practice notebook. For the result

In [1]:
import pandas as pd
import sqlite3
from sqlalchemy.engine import create_engine
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn import preprocessing
from time import time
from utils import featurize_date_col, is_russian_2017_holiday

pd.options.mode.chained_assignment = None
%matplotlib inline

In [2]:
# avito = sqlite3.connect("data/avito.db")
start_time = time()
# either:
# 1) (only the read_csv parts take 30 seconds)
train = pd.read_csv('data/periods_train.csv')
# train.to_sql('periods_train', avito)
test = pd.read_csv('data/periods_test.csv')
# test.to_sql('periods_test', avito)

# or 2) (it took 120 seconds)
# train = pd.read_sql('SELECT * from periods_train', avito, index_col='item_id')
# test = pd.read_sql('SELECT * from periods_test', avito, index_col='item_id')

print('it took {} seconds'.format(time() - start_time))

it took 31.845133066177368 seconds


In [3]:
train[1000000:1500000].to_csv('data/periods_train_500000.csv')

In [4]:
train.head()

,item_id,activation_date,date_from,date_to
0,8f5caef7afb0,2017-02-14,2017-03-15,2017-03-16
1,66218ff526d1,2017-02-16,2017-03-15,2017-03-18
2,b237d9539b21,2017-03-01,2017-03-15,2017-03-28
3,80bf58082ad3,2017-03-19,2017-03-19,2017-03-28
4,67a9944a7373,2017-03-14,2017-03-15,2017-03-28


In [5]:
df = train[1000000:1500000]

In [6]:
null_idx = df['activation_date'].isnull()
df['activation_date'].loc[null_idx] = df['date_from'].loc[null_idx]

In [7]:
len(df['item_id'].unique())

497384

In [8]:
temp = df[df['item_id'].duplicated()]

In [9]:
dft = df[df['item_id'].isin(temp['item_id'])]

In [10]:
observe_ids = dft['item_id']

## initial preparation and sanity checks

In [11]:
for col in ['activation_date', 'date_from', 'date_to']:
    df[col] = pd.to_datetime(df[col], format='%Y-%m-%d')

In [12]:
assert (df['date_to'] >= df['date_from']).all()

In [13]:
assert (df['date_from'] >= df['activation_date']).all()

## Feature generation

In [14]:
df['days_to_publish'] = (df['date_from'] - df['activation_date']).dt.days

In [15]:
df['days_online'] = (df['date_to'] - df['date_from']).dt.days

In [16]:
for col in ['activation_date', 'date_from', 'date_to']:
    df = featurize_date_col(df, col, remove_when_done=False)

In [17]:
df.head()

,item_id,activation_date,date_from,date_to,days_to_publish,days_online,activation_date_isholiday,activation_date_wday,activation_date_yday,date_from_isholiday,date_from_wday,date_from_yday,date_to_isholiday,date_to_wday,date_to_yday
1000000,b8a335c6ceee,2017-03-10,2017-03-15,2017-03-28,5,13,0,4,69,0,2,74,0,1,87
1000001,e208f79e7c47,2017-02-27,2017-03-15,2017-03-18,16,3,0,0,58,0,2,74,0,5,77
1000002,87c311723851,2017-03-19,2017-03-19,2017-03-28,0,9,0,6,78,0,6,78,0,1,87
1000003,6f650813ed85,2017-03-14,2017-03-15,2017-03-28,1,13,0,1,73,0,2,74,0,1,87
1000004,0b2fc250404f,2017-03-20,2017-03-20,2017-03-28,0,8,0,0,79,0,0,79,0,1,87


In [22]:
grouped = df.groupby('item_id')

In [23]:
base = grouped[['item_id']].count().rename(columns={'item_id': 'nlisted'})

In [24]:
base['sum_days_online'] = grouped[['days_online']].sum()
base['mean_days_online'] = grouped[['days_online']].mean()
base['last_days_online'] = grouped[['days_online']].last()
base['sum_days_to_publishe'] = grouped[['days_to_publish']].sum()
base['mean_days_to_publish'] = grouped[['days_to_publish']].mean()
base['median_date_to_isholiday'] = grouped[['date_to_isholiday']].median()
base['median_date_to_wday'] = grouped[['date_to_wday']].median()
base['median_date_to_yday'] = grouped[['date_to_yday']].median()

base['start_date'] = grouped[['date_from']].min()
base['end_date'] = grouped[['date_to']].max()
for col in ['start_date', 'end_date']:
    base = featurize_date_col(base, col, remove_when_done=True)

In [25]:
base.loc[observe_ids].head()

,nlisted,sum_days_online,mean_days_online,last_days_online,sum_days_to_publishe,mean_days_to_publish,median_date_to_isholiday,median_date_to_wday,median_date_to_yday,start_date_isholiday,start_date_wday,start_date_yday,end_date_isholiday,end_date_wday,end_date_yday
item_id,,,,,,,,,,,,,,,
add31fa463ff,2,12,6.0,7,22,11.0,0,1.5,84.0,0,2,74,0,1,87
c45babdedc70,2,7,3.5,6,29,14.5,0,2.0,81.0,0,2,74,0,1,87
d3b49ff66adb,2,12,6.0,4,22,11.0,0,2.0,84.5,0,2,74,0,1,87
6988ea13f791,2,12,6.0,7,25,12.5,0,0.5,83.0,0,2,74,0,1,87
7c0563f4d6b0,2,12,6.0,5,23,11.5,0,1.5,84.0,0,2,74,0,1,87
